# Tile z and zo 
Tile the ASCII text files containing x, y, z and zo information.

In [1]:
import pdal
import json
import geopandas
import shapely
import shapely.wkt
import pathlib
import shutil 
import rioxarray
import numpy

## Copy locally

In [2]:
crs = 32758
z_low_noise = -10

In [3]:
# Setup remote and local paths
remote_path = pathlib.Path(r"\\niwa.local\projects\hamilton\GNS23201\Working\Flooding\Roughness\Z_Zo_Tiles")
local_path = pathlib.Path(r"C:\Local\data\GNS23201\tiles")
(local_path / "raw").mkdir(exist_ok = True)
(local_path / "las").mkdir(exist_ok = True)
(local_path / "zo").mkdir(exist_ok = True)

In [4]:
for file in remote_path.iterdir():
    #print(f"Copy {file.name} locally") 
    shutil.copy2(file, local_path / "raw" / file.name)

# Z processing
## Convert to LAS

In [5]:
# Convert each to las
for file in (local_path / "raw").iterdir():
    #print(f"Converting {file.name} to LAS")
    pdal_pipeline_instructions = [
        {
            "type": "readers.text",
            "filename": str(file),
            "header": "X, Y, Z, Intensity"
        },
        {
            "type":"writers.las",
            "filename":str(file.parent.parent / "las" / f"{file.stem}.las"),
            "a_srs": f"EPSG:{crs}"
        }
    ]
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()

## Combine into a single LAZ

In [ ]:
# Create instruction
pdal_pipeline_instructions = []
for file in (local_path / "las").iterdir():
    if file.suffix == '.las':
        pdal_pipeline_instructions.append(
            {"type": "readers.las",
            "filename": str(file),})
pdal_pipeline_instructions.append({"type": "filters.merge",})
pdal_pipeline_instructions.append(
    {"type":"writers.las",
     "filename": str(local_path / "las" / "combined.laz"),
     "a_srs": f"EPSG:{crs}",
     "compression": "laszip"})
# Run pipeline
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

## Remove noise points

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "las" / "combined.laz"), }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();
# Remove noise
points = pdal_pipeline.arrays[0]
points = points[points['Z'] > z_low_noise]
# Write out value
pdal_pipeline_instructions = [
            {"type": "writers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"),
             "compression": "laszip",
             "a_srs": f"EPSG:{crs}"
            }
        ]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [points])
pdal_pipeline.execute()
# Write out crs information
dem = rioxarray.open_rasterio(local_path / "las" / "combined.tif")
dem = dem.rio.write_crs(crs)
dem.rio.to_raster(local_path / "las" / "combined.tif")

In [ ]:
pdal_pipeline_instructions = [
            {"type": "writers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"),
                "compression": "laszip",
            }
        ]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [points])
pdal_pipeline.execute()

## Rasterise at 1m and save as a Geotiff

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"), },
    { "type": "filters.delaunay" },
    { "type": "filters.faceraster", "resolution": 1 },
    { "type": "writers.raster", "filename": str(local_path / "las" / "combined.tif") }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

# zo processing
## Save a LAZ with zo as Z

In [ ]:
# Convert each to las
for file in (local_path / "raw").iterdir():
    #print(f"Converting {file.name} to LAS")
    pdal_pipeline_instructions = [
        {
            "type": "readers.text",
            "filename": str(file),
            "header": "X, Y, zo, Z"
        },
        {
            "type":"writers.las",
            "filename":str(file.parent.parent / "zo" / f"{file.stem}.las"),
            "a_srs": f"EPSG:{crs}"
        }
    ]
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()

## Combine into a single LAZ

In [ ]:
# Create instruction
pdal_pipeline_instructions = []
for file in (local_path / "zo").iterdir():
    if file.suffix == '.las':
        pdal_pipeline_instructions.append(
            {"type": "readers.las",
            "filename": str(file),})
pdal_pipeline_instructions.append({"type": "filters.merge",})
pdal_pipeline_instructions.append(
    {"type":"writers.las",
     "filename": str(local_path / "zo" / "combined.laz"),
     "compression": "laszip"})
# Run pipeline
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

## Rasterise 1m and save as a Geotiff

## Get catchment outline

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "zo" / "combined.laz"), },
    { "type": "filters.delaunay" },
    { "type": "filters.faceraster", "resolution": 1 },
    { "type": "writers.raster", "filename": str(local_path / "zo" / "combined.tif") }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();
# Write out crs information
dem = rioxarray.open_rasterio(local_path / "zo" / "combined.tif")
dem = dem.rio.write_crs(crs)
dem.rio.to_raster(local_path / "zo" / "combined.tif")

# Remove outliars, re-interpolate and clip
## Get boundary

## Read in Geotiffs and clip

In [ ]:
polygon = geopandas.read_file(local_path / "combined_boundary_manual_crs.geojson")
dem = rioxarray.open_rasterio(local_path / "las" / "combined.tif")
clipped = dem.rio.clip(polygon.geometry)
clipped.rio.to_raster(local_path / "las" / "combined_clipped.tif")

In [ ]:
zo = rioxarray.open_rasterio(local_path / "zo" / "combined.tif")
clipped = zo.rio.clip(polygon.geometry)
clipped.rio.to_raster(local_path / "zo" / "combined_clipped.tif")

## Add a background zo layer to the lower resolution DEM

In [35]:
zo_land = 0.2
zo_ocean = 0.004

zo = rioxarray.open_rasterio(local_path / "5m_dem_espiritu.nc").squeeze("band", drop=True)
zo.data[numpy.logical_not(numpy.isnan(zo.data))] = zo_land
zo.data[numpy.isnan(zo.data)] = zo_ocean
zo=zo.assign_attrs(STATISTICS_MAXIMUM=zo.data.max())
zo=zo.assign_attrs(STATISTICS_MEAN=zo.data.mean())
zo=zo.assign_attrs(STATISTICS_MINIMUM=zo.data.min())
zo=zo.assign_attrs(STATISTICS_VALID_PERCENT=100)
zo.to_netcdf(local_path / "5m_zo_espiritu.nc", format="NETCDF4", engine="netcdf4")